In [22]:
# 基础模块
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from datetime import datetime
from collections import defaultdict

# 可视化配置
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'STHeiti'         # 中文支持（macOS）
plt.rcParams['axes.unicode_minus'] = False      # 正负号支持
%matplotlib inline

# 轨迹聚类与建模
from sklearn.cluster import DBSCAN
from pyproj import Transformer

# API调用（如POI增强）
import requests
import time
import json

# Neo4j 图数据库
from py2neo import Graph, Node, Relationship  # 若报错先注释，等后面阶段再装
from neo4j import GraphDatabase

# 路径配置
base_dir = os.path.dirname(os.path.abspath("__file__"))  # 当前脚本所在目录
root_dir = os.path.abspath(os.path.join(base_dir, '..'))  # 项目根目录
output_dir = os.path.join(root_dir, 'outputs')
traj_path = os.path.join(output_dir, 'geolife_cleaned_traj.csv')

In [3]:
# Step 1：加载数据
df = pd.read_csv(traj_path)
df['t'] = pd.to_datetime(df['t'])

# Step 2：空间聚类识别热点节点
eps = 0.0006      # 空间阈值（近似50米）
min_samples = 5
db = DBSCAN(eps=eps, min_samples=min_samples)
df['node'] = db.fit_predict(df[['x', 'y']])

# 移除噪声节点
df = df[df['node'] != -1].reset_index(drop=True)

# Step 3：构建轨迹节点序列 & 时间序列
traj_records = []

for (uid, traj_id), group in df.groupby(['uid', 'traj_id']):
    group = group.sort_values('t')
    nodes = group['node'].tolist()
    times = group['t'].tolist()

    # 去除连续重复节点
    clean_nodes = [nodes[0]]
    clean_times = [times[0]]
    for i in range(1, len(nodes)):
        if nodes[i] != clean_nodes[-1]:
            clean_nodes.append(nodes[i])
            clean_times.append(times[i])

    if len(clean_nodes) >= 2:
        traj_records.append({
            'uid': int(uid),
            'traj_id': int(traj_id),
            'start_time': clean_times[0].time(),
            'end_time': clean_times[-1].time(),
            'node_sequence': json.dumps(clean_nodes, ensure_ascii=False),
            'time_sequence': json.dumps([t.strftime("%H:%M:%S") for t in clean_times], ensure_ascii=False)
        })

print(f"构建完成，共记录轨迹数：{len(traj_records)}")

构建完成，共记录轨迹数：1181


In [4]:
traj_meta_df = pd.DataFrame(traj_records)
metadata_path = os.path.join(output_dir, 'traj_metadata.csv')
traj_meta_df.to_csv(metadata_path, index=False)

print(f"轨迹元数据表已保存：{metadata_path}")

轨迹元数据表已保存：/Users/chenenqiang/Desktop/Undergraduate Life/Undergraduate Life/创新实验2025春/FrequentPatternMiningBasedOnHotspotTrajectories/DataPreprocess/outputs/traj_metadata.csv


In [5]:
# 路径配置
base_dir = os.path.dirname(os.path.abspath("__file__"))
root_dir = os.path.abspath(os.path.join(base_dir, '..'))
output_dir = os.path.join(root_dir, 'outputs')
meta_path = os.path.join(output_dir, 'traj_metadata.csv')

# 加载轨迹元数据
df = pd.read_csv(meta_path)
df['node_sequence'] = df['node_sequence'].apply(json.loads)

# 构建N度路径表：SN → SG集合（uid, traj_id）
ndegree_path_table = defaultdict(set)

for _, row in df.iterrows():
    uid = int(row['uid'])
    traj_id = int(row['traj_id'])
    nodes = row['node_sequence']
    path_len = len(nodes)

    for n in range(1, path_len):  # 枚举所有n阶路径（至少2个节点）
        for i in range(path_len - n):
            subpath = tuple(nodes[i:i + n + 1])  # 如 [1,2], [2,3,4]
            ndegree_path_table[subpath].add((uid, traj_id))

print(f"N度路径构建完成，共有唯一路径：{len(ndegree_path_table)} 条")

# 输出为标准CSV结构：SN, SG, k
ndegree_path_df = pd.DataFrame([
    {
        'SN': json.dumps(list(path), ensure_ascii=False),
        'SG': json.dumps([list(pair) for pair in sorted(sg_set)], ensure_ascii=False),
        'k': len(path)
    }
    for path, sg_set in ndegree_path_table.items()
])

output_file = os.path.join(output_dir, 'ndegree_path_table.csv')
ndegree_path_df.to_csv(output_file, index=False)

print(f"N度路径表已保存至：{output_file}")

N度路径构建完成，共有唯一路径：5970 条
N度路径表已保存至：/Users/chenenqiang/Desktop/Undergraduate Life/Undergraduate Life/创新实验2025春/FrequentPatternMiningBasedOnHotspotTrajectories/DataPreprocess/outputs/ndegree_path_table.csv


In [6]:
# 加载原始轨迹数据
df = pd.read_csv(traj_path)

# DBSCAN聚类参数（保持与之前完全一致）
eps = 0.0006
min_samples = 5
db = DBSCAN(eps=eps, min_samples=min_samples)
df['node'] = db.fit_predict(df[['x', 'y']])
df = df[df['node'] != -1]  # 移除噪声

# 计算每个 node_id 的坐标中心
node_coords = df.groupby('node')[['x', 'y']].mean().reset_index()
node_coords.columns = ['node_id', 'x', 'y']

# 保存为 nodes.csv
nodes_path = os.path.join(output_dir, 'nodes.csv')
node_coords.to_csv(nodes_path, index=False)
print(f"节点中心文件已保存至：{nodes_path}")

节点中心文件已保存至：/Users/chenenqiang/Desktop/Undergraduate Life/Undergraduate Life/创新实验2025春/FrequentPatternMiningBasedOnHotspotTrajectories/DataPreprocess/outputs/nodes.csv


In [8]:
# 读取 traj_metadata
meta_path = os.path.join(output_dir, 'traj_metadata.csv')
meta_df = pd.read_csv(meta_path)
meta_df['node_sequence'] = meta_df['node_sequence'].apply(json.loads)

# 构建边频率与所属轨迹集合
edge_freq = defaultdict(int)
edge_trajs = defaultdict(set)

for _, row in meta_df.iterrows():
    uid, traj_id = int(row['uid']), int(row['traj_id'])
    nodes = row['node_sequence']

    for i in range(len(nodes) - 1):
        edge = (nodes[i], nodes[i+1])
        edge_freq[edge] += 1
        edge_trajs[edge].add((uid, traj_id))

# 构建输出表
edges_df = pd.DataFrame([
    {
        'source': src,
        'target': tgt,
        'frequency': edge_freq[(src, tgt)],
        'traj_ids': json.dumps(sorted([list(x) for x in edge_trajs[(src, tgt)]]), ensure_ascii=False)
    }
    for (src, tgt) in edge_freq
])

# 保存为 edges.csv
edges_path = os.path.join(output_dir, 'edges.csv')
edges_df.to_csv(edges_path, index=False)
print(f"边文件已保存至：{edges_path}")

边文件已保存至：/Users/chenenqiang/Desktop/Undergraduate Life/Undergraduate Life/创新实验2025春/FrequentPatternMiningBasedOnHotspotTrajectories/DataPreprocess/outputs/edges.csv


In [12]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "#020728Ceq"

driver = GraphDatabase.driver(uri, auth=(user, password))

# 清空整个数据库（慎用）
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

print("Neo4j 已清空所有节点和关系。")

driver.close()

Neo4j 已清空所有节点和关系。


In [13]:
# 路径配置
base_dir = os.path.dirname(os.path.abspath("__file__"))
root_dir = os.path.abspath(os.path.join(base_dir, '..'))
output_dir = os.path.join(root_dir, 'outputs')
nodes_path = os.path.join(output_dir, 'nodes.csv')
edges_path = os.path.join(output_dir, 'edges.csv')

# Neo4j 连接信息
uri = "bolt://localhost:7687"
user = "neo4j"
password = "#020728Ceq"

driver = GraphDatabase.driver(uri, auth=(user, password))

# 节点导入函数
def import_node(tx, node_id, x, y):
    tx.run("""
        MERGE (n:Hotspot {id: $node_id})
        SET n.x = $x, n.y = $y
    """, node_id=node_id, x=x, y=y)

# 边导入函数
def import_edge(tx, source, target, frequency, traj_ids_flat):
    tx.run("""
        MATCH (a:Hotspot {id: $source})
        MATCH (b:Hotspot {id: $target})
        MERGE (a)-[r:TRAJ_EDGE]->(b)
        SET r.frequency = $frequency,
            r.traj_ids = $traj_ids
    """, source=source, target=target, frequency=frequency, traj_ids=traj_ids_flat)

with driver.session() as session:
    print("导入节点中...")
    nodes_df = pd.read_csv(nodes_path)
    for _, row in nodes_df.iterrows():
        session.execute_write(import_node, int(row['node_id']), float(row['x']), float(row['y']))

    print("导入边中...")
    edges_df = pd.read_csv(edges_path)
    for _, row in edges_df.iterrows():
        # 修复嵌套数组问题：将 [[1,2],[2,3]] → ["1_2", "2_3"]
        raw_traj_ids = json.loads(row['traj_ids']) if isinstance(row['traj_ids'], str) else []
        traj_ids_flat = [f"{uid}_{tid}" for uid, tid in raw_traj_ids]

        session.execute_write(
            import_edge,
            int(row['source']),
            int(row['target']),
            int(row['frequency']),
            traj_ids_flat
        )

driver.close()
print("Neo4j 数据导入完成")

导入节点中...
导入边中...
Neo4j 数据导入完成


In [40]:
# ================= 工具函数 =================
def safe_parse_json_list(x):
    if isinstance(x, str):
        return json.loads(x)
    return x


def preprocess_ndegree_df(df):
    df['SN'] = df['SN'].apply(safe_parse_json_list)
    df['SG'] = df['SG'].apply(lambda x: set(tuple(i) for i in safe_parse_json_list(x)))
    return df


def save_result(df, filename, output_dir='outputs'):
    os.makedirs(output_dir, exist_ok=True)

    # 强制转换 path 为 JSON 字符串格式，防止 tuple 存储出错
    if 'path' in df.columns:
        df['path'] = df['path'].apply(lambda x: json.dumps(list(x)) if isinstance(x, (list, tuple)) else x)

    out_path = os.path.join(output_dir, filename)
    df.to_csv(out_path, index=False)
    print(f"结果已保存到: {out_path}")



# ================= NDTTJ 算法 =================
def run_ndttj(ndegree_df, m=5, k=2, save_as=None):
    ndegree_df = preprocess_ndegree_df(ndegree_df)
    ndttj_result = []
    path_table = defaultdict(list)
    for row in ndegree_df.itertuples():
        if len(row.SG) >= m:
            path_table[row.k].append((tuple(row.SN), row.SG))

    final_result = {}
    for ki in sorted(path_table.keys()):
        for p1, sg1 in path_table[ki]:
            for p2, sg2 in path_table[ki]:
                if p1[1:] == p2[:-1]:
                    new_path = p1 + (p2[-1],)
                    if new_path in final_result:
                        continue
                    new_sg = sg1 & sg2
                    if len(new_sg) >= m and len(new_path) >= k:
                        final_result[new_path] = new_sg

    for path, sg in final_result.items():
        ndttj_result.append({
            'path': path,
            'frequency': len(sg),
            'traj_ids': json.dumps(sorted(list(sg)), ensure_ascii=False)
        })

    df_result = pd.DataFrame(ndttj_result)
    if save_as:
        save_result(df_result, save_as)
    return df_result


# ================= NDTTT 算法 =================
def run_ndttt(ndegree_df, m=5, k=2, save_as=None):
    ndegree_df = preprocess_ndegree_df(ndegree_df)
    ndttt_result = []
    path_dict = {tuple(row.SN): row.SG for row in ndegree_df.itertuples() if len(row.SG) >= m}
    visited = set()

    def extend(path, sg):
        results = []
        for p in path_dict:
            if len(p) == len(path) + 1 and p[:-1] == path:
                new_sg = sg & path_dict[p]
                if len(new_sg) >= m and len(p) >= k and p not in visited:
                    visited.add(p)
                    results.append((p, new_sg))
                    results += extend(p, new_sg)
        return results

    for path, sg in path_dict.items():
        if len(path) >= k:
            results = extend(path, sg)
            for p, sg_p in results:
                ndttt_result.append({
                    'path': p,
                    'frequency': len(sg_p),
                    'traj_ids': json.dumps(sorted(list(sg_p)), ensure_ascii=False)
                })

    df_result = pd.DataFrame(ndttt_result)
    if save_as:
        save_result(df_result, save_as)
    return df_result


# ================= TTHS 算法 =================
def run_tths_from_neo4j(uri, user, password, m=5, k=2, save_as=None):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    results = []
    visited_paths = set()

    def dfs(tx, path, traj_ids):
        if len(path) >= k and len(traj_ids) >= m:
            key = tuple(path)
            if key not in visited_paths:
                visited_paths.add(key)
                results.append({
                    'path': path[:],
                    'frequency': len(traj_ids),
                    'traj_ids': json.dumps(sorted(list(traj_ids)), ensure_ascii=False)
                })
        if len(path) > 12:
            return

        query = """
        MATCH (n:Hotspot {id: $nid})-[r:TRAJ_EDGE]->(m)
        RETURN m.id AS next_id, r.traj_ids AS tids
        """
        result = tx.run(query, nid=path[-1])
        for record in result:
            next_id = record['next_id']
            if next_id in path:
                continue
            tids = set(tuple(map(int, tid.split('_'))) for tid in record['tids'])
            intersected = traj_ids & tids
            if len(intersected) >= m:
                dfs(tx, path + [next_id], intersected)

    with driver.session() as session:
        start_nodes = session.run("MATCH (n:Hotspot) RETURN n.id AS nid")
        for record in start_nodes:
            nid = record['nid']
            edges = session.run("""
                MATCH (n:Hotspot {id: $nid})-[r:TRAJ_EDGE]->(m)
                RETURN m.id AS next_id, r.traj_ids AS tids
            """, nid=nid)
            for edge in edges:
                next_id = edge['next_id']
                tids = set(tuple(map(int, tid.split('_'))) for tid in edge['tids'])
                if len(tids) >= m:
                    dfs(session, [nid, next_id], tids)

    driver.close()
    df_result = pd.DataFrame(results)
    if save_as:
        save_result(df_result, save_as)
    return df_result

In [41]:
ndegree_path_file = os.path.join(output_dir, "ndegree_path_table.csv")
ndegree_df = pd.read_csv(ndegree_path_file)

# 执行三个算法
ndttj_df = run_ndttj(ndegree_df, m=5, k=2, save_as="ndttj_hotspot_paths.csv")
ndttt_df = run_ndttt(ndegree_df, m=5, k=2, save_as="ndttt_hotspot_paths.csv")
tths_df  = run_tths_from_neo4j(
    uri="bolt://localhost:7687",
    user="neo4j",
    password="#020728Ceq",
    m=5, k=2,
    save_as="tths_hotspot_paths.csv"
)

print("NDTTJ 示例结果：")
print(ndttj_df.head())

print("NDTTT 示例结果：")
print(ndttt_df.head())

print("TTHS  示例结果：")
print(tths_df.head())

结果已保存到: outputs/ndttj_hotspot_paths.csv
结果已保存到: outputs/ndttt_hotspot_paths.csv
结果已保存到: outputs/tths_hotspot_paths.csv
NDTTJ 示例结果：
          path  frequency                                           traj_ids
0  [0, 1, 116]          6  [[35, 9], [35, 13], [35, 23], [35, 24], [35, 3...
1    [1, 2, 1]          6  [[1, 16], [1, 35], [1, 36], [1, 62], [1, 64], ...
2    [3, 0, 3]          7  [[1, 13], [5, 18], [5, 27], [5, 71], [96, 12],...
3    [2, 1, 2]          6  [[1, 16], [1, 35], [1, 36], [1, 62], [1, 64], ...
4    [2, 1, 0]          5      [[1, 28], [1, 31], [1, 59], [1, 63], [1, 69]]
NDTTT 示例结果：
          path  frequency                                           traj_ids
0  [0, 1, 116]          6  [[35, 9], [35, 13], [35, 23], [35, 24], [35, 3...
1    [2, 1, 0]          5      [[1, 28], [1, 31], [1, 59], [1, 63], [1, 69]]
2   [2, 1, 23]          7  [[1, 35], [1, 36], [1, 39], [1, 45], [1, 62], ...
3   [0, 34, 0]          6  [[5, 6], [96, 1], [96, 12], [96, 49], [179, 25...
4   [0, 33

In [42]:
from ast import literal_eval

def safe_eval_traj_ids(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    try:
        return literal_eval(x)
    except Exception:
        return []

# === 安全解析 path ===
def safe_parse_path(x):
    if pd.isna(x):
        return ()
    if isinstance(x, list):
        return tuple(x)
    if isinstance(x, str) and x.strip().startswith('['):
        try:
            return tuple(json.loads(x))
        except Exception:
            return ()
    return ()

# === 加载数据并添加 source 标签 ===
def load_with_source(path, source_name):
    df = pd.read_csv(path)
    df['path'] = df['path'].apply(safe_parse_path)
    df['traj_ids'] = df['traj_ids'].apply(safe_eval_traj_ids)
    df['source'] = [[source_name]] * len(df)
    df = df[df['path'].apply(lambda x: len(x) > 0)]  # ✅ 去除空路径
    return df

# === 加载三个文件 ===
ndttj_df = load_with_source(os.path.join(output_dir, 'ndttj_hotspot_paths.csv'), 'NDTTJ')
ndttt_df = load_with_source(os.path.join(output_dir, 'ndttt_hotspot_paths.csv'), 'NDTTT')
tths_df  = load_with_source(os.path.join(output_dir, 'tths_hotspot_paths.csv'),  'TTHS')

# === 合并前统计
print("NDTTJ:", len(ndttj_df), "NDTTT:", len(ndttt_df), "TTHS:", len(tths_df))

# === 合并并聚合 ===
merged_df = pd.concat([ndttj_df, ndttt_df, tths_df], ignore_index=True)

def merge_groups(group):
    merged_traj_ids = {tuple(x) for sublist in group['traj_ids'] for x in sublist}
    merged_sources = sorted(set(src for sources in group['source'] for src in sources))
    return pd.Series({
        'frequency': max(group['frequency']),
        'traj_ids': json.dumps(sorted(merged_traj_ids)),
        'source': merged_sources
    })

merged_df = merged_df.groupby('path', group_keys=False).apply(merge_groups).reset_index()
merged_df['path'] = merged_df['path'].apply(list)  # 输出为 JSON

# === 统计来源数量
print("路径来源统计：")
print(merged_df['source'].explode().value_counts())

# === 输出文件 ===
output_path = os.path.join(output_dir, 'merged_hotspot_paths.csv')
merged_df.to_csv(output_path, index=False)
print(f"合并完成，输出文件已保存：{output_path}")

NDTTJ: 65 NDTTT: 45 TTHS: 166
路径来源统计：
source
TTHS     166
NDTTJ     65
NDTTT     45
Name: count, dtype: int64
合并完成，输出文件已保存：/Users/chenenqiang/Desktop/Undergraduate Life/Undergraduate Life/创新实验2025春/FrequentPatternMiningBasedOnHotspotTrajectories/DataPreprocess/outputs/merged_hotspot_paths.csv


/var/folders/66/j4x4v1zx3w501lq05cqk8g8h0000gn/T/ipykernel_3258/3015770336.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('path', group_keys=False).apply(merge_groups).reset_index()
